In [10]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

<h1>XGBoost Cloud Prediction Invocation Template</h1>
<h4>Invoke SageMaker Prediction Service</h4>

In [11]:
import boto3
import re
from sagemaker import get_execution_role
import sagemaker

In [12]:
# Acquire a realtime endpoint
endpoint_name = 'xgboost-bikerental-v1'
predictor = sagemaker.predictor.RealTimePredictor(endpoint_name=endpoint_name)

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [13]:
from sagemaker.serializers import CSVSerializer
predictor.serializer = CSVSerializer()

In [14]:
df_all = pd.read_csv('bike_test.csv')

In [15]:
df_all.head()

datetime  season  holiday  workingday  weather   temp   atemp  \
0  2011-01-20 00:00:00       1        0           1        1  10.66  11.365   
1  2011-01-20 01:00:00       1        0           1        1  10.66  13.635   
2  2011-01-20 02:00:00       1        0           1        1  10.66  13.635   
3  2011-01-20 03:00:00       1        0           1        1  10.66  12.880   
4  2011-01-20 04:00:00       1        0           1        1  10.66  12.880   

   humidity  windspeed  year  month  day  dayofweek  hour  
0        56    26.0027  2011      1   20          3     0  
1        56     0.0000  2011      1   20          3     1  
2        56     0.0000  2011      1   20          3     2  
3        56    11.0014  2011      1   20          3     3  
4        56    11.0014  2011      1   20          3     4

In [16]:
df_all.columns[1:]

Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek', 'hour'],
      dtype='object')

In [17]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1],[20,1]]
arr_test = df_all[df_all.columns[1:]].values

In [18]:
type(arr_test)

numpy.ndarray

In [19]:
arr_test.shape

(6493, 13)

In [20]:
arr_test[:5]

array([[1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01,
        1.13650e+01, 5.60000e+01, 2.60027e+01, 2.01100e+03, 1.00000e+00,
        2.00000e+01, 3.00000e+00, 0.00000e+00],
       [1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01,
        1.36350e+01, 5.60000e+01, 0.00000e+00, 2.01100e+03, 1.00000e+00,
        2.00000e+01, 3.00000e+00, 1.00000e+00],
       [1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01,
        1.36350e+01, 5.60000e+01, 0.00000e+00, 2.01100e+03, 1.00000e+00,
        2.00000e+01, 3.00000e+00, 2.00000e+00],
       [1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01,
        1.28800e+01, 5.60000e+01, 1.10014e+01, 2.01100e+03, 1.00000e+00,
        2.00000e+01, 3.00000e+00, 3.00000e+00],
       [1.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 1.06600e+01,
        1.28800e+01, 5.60000e+01, 1.10014e+01, 2.01100e+03, 1.00000e+00,
        2.00000e+01, 3.00000e+00, 4.00000e+00]])

In [21]:
result = predictor.predict(arr_test[:2])

In [22]:
result

b'10.033907890319824,-1.0149911642074585'

In [23]:
arr_test.shape

(6493, 13)

In [24]:
# For large number of predictions, we can split the input data and
# Query the prediction service.
# array_split is convenient to specify how many splits are needed
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    result = result.decode("utf-8")
    result = result.split(',')
    print (arr.shape)
    predictions += [float(r) for r in result]

(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)


In [25]:
len(predictions)

6493

In [26]:
np.expm1(predictions)

/tmp/ipykernel_25070/2913177442.py:1: RuntimeWarning: overflow encountered in expm1
  np.expm1(predictions)


array([ 2.27851435e+04, -6.37594368e-01, -9.96990228e-01, ...,
        3.63590654e+66,  5.68933749e+37,  1.42578436e+17])

In [27]:
df_all['count'] = np.expm1(predictions)

/tmp/ipykernel_25070/2952958062.py:1: RuntimeWarning: overflow encountered in expm1
  df_all['count'] = np.expm1(predictions)


In [28]:
df_all.head()

datetime  season  holiday  workingday  weather   temp   atemp  \
0  2011-01-20 00:00:00       1        0           1        1  10.66  11.365   
1  2011-01-20 01:00:00       1        0           1        1  10.66  13.635   
2  2011-01-20 02:00:00       1        0           1        1  10.66  13.635   
3  2011-01-20 03:00:00       1        0           1        1  10.66  12.880   
4  2011-01-20 04:00:00       1        0           1        1  10.66  12.880   

   humidity  windspeed  year  month  day  dayofweek  hour         count  
0        56    26.0027  2011      1   20          3     0  22785.143531  
1        56     0.0000  2011      1   20          3     1     -0.637594  
2        56     0.0000  2011      1   20          3     2     -0.996990  
3        56    11.0014  2011      1   20          3     3     -0.991257  
4        56    11.0014  2011      1   20          3     4     -0.991257

In [29]:
df_all[['datetime','count']].to_csv('predicted_count_cloud.csv',index=False)